In [1]:
pip install ortools

  Using cached ortools-9.8.3296-cp310-cp310-win_amd64.whl (101.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Importamos el módulo pywraplp de ortools, que nos ayudara con herramientas para la creación y solución de problemas de programación lineal.
from ortools.linear_solver import pywraplp
import random

# Definimos una función para resolver el problema de asignación de ONUs.
def solve_onu_assignment(num_onus, num_users, U, distances):
    # Creamos un nuevo solucionador utilizando el backend SCIP, que es uno de los solucionadores disponibles en ortools.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    
    # Si el solucionador no se encuentra, lanzamos una excepción.
    if not solver:
        raise Exception('Solver not found.')
    
    # Creamos un diccionario para almacenar las variables de decisión. 
    # Cada variable representa si un usuario u está conectado a una ONU o (1 si está conectado, 0 si no).
    x = {}
    for o in range(num_onus):
        for u in range(num_users):
            x[o, u] = solver.BoolVar(f'x[{o},{u}]')
    
    # Se define la función objetivo, que es minimizar el costo total.
    solver.Minimize(solver.Sum([distances[o, u] * x[o, u] for o in range(num_onus) for u in range(num_users)]))
    
    # Agregamos restricciones para asegurar que cada usuario esté conectado exactamente a una ONU.
    for u in range(num_users):
        solver.Add(solver.Sum([x[o, u] for o in range(num_onus)]) == 1)
    
    # Agregamos restricciones para asegurar que cada ONU no maneje más de U usuarios.
    for o in range(num_onus):
        solver.Add(solver.Sum([x[o, u] for u in range(num_users)]) <= U)
    
    # Solicitamos al solucionador que encuentre una solución óptima.
    status = solver.Solve()
    
    # Evaluamos el estado de la solución. Si es óptima, imprimimos el valor de la función objetivo
    # y la asignación de usuarios a ONUs. Si no hay solución óptima, informamos al usuario.
    if status == pywraplp.Solver.OPTIMAL:
        print('Total cost =', solver.Objective().Value())
        for o in range(num_onus):
            for u in range(num_users):
                if x[o, u].solution_value() > 0:
                    print(f'User {u} is connected to ONU {o}')
    else:
        print('The problem does not have an optimal solution.')

# Se plantea un ejemplo para utilizar la función.
# Se stablecen el número de ONUs y usuarios, la capacidad máxima de usuarios por ONU,
# ademas de un diccionario de distancias.

# Ejemplo para utilizar la función.
num_onus = 20 # Número de ONUs
num_users = 10 # Número de usuarios
U = 2 # Número máximo de usuarios por ONU

# Se crea un diccionario de distancias vacío.
distances = {}

# Se puede rellenar este diccionario 'distances' con los valores que se deseen utilizar.
# Función que calcula las distancias basándose en datos de entrada.
for o in range(num_onus):
    for u in range(num_users):
        # Aquí se asignan las distancias aleatoria. Se propone un ejemplo entre 1 y 10.
        distances[(o, u)] = random.randint(1, 10) 

# Se llama a la función con el diccionario de distancias como parámetro para poder generar la solucion.
solve_onu_assignment(num_onus, num_users, U, distances)

Total cost = 11.0
User 4 is connected to ONU 1
User 1 is connected to ONU 2
User 3 is connected to ONU 2
User 7 is connected to ONU 3
User 9 is connected to ONU 5
User 0 is connected to ONU 6
User 8 is connected to ONU 7
User 2 is connected to ONU 8
User 6 is connected to ONU 11
User 5 is connected to ONU 15
